In [0]:
class OurQueue:
    """
    A queue for counting efficiently the number of events within time windows.
    Complexity:
        All operators in amortized O(W) time where W is the number of windows.
    """
    def __init__(self):
        self.now = None
        self.queue = []
        self.window_lengths = [3600 * 24 * 30, 3600 * 24 * 7, 3600 * 24, 3600]
        self.cursors = [0] * len(self.window_lengths)

    def __len__(self):
        return len(self.queue)

    def get_counters(self, t):
        self.update_cursors(t)
        return [len(self.queue)] + [len(self.queue) - cursor
                                    for cursor in self.cursors]

    def push(self, time):
        self.queue.append(time)

    def update_cursors(self, t):
        for pos, length in enumerate(self.window_lengths):
            while (self.cursors[pos] < len(self.queue) and
                   t - self.queue[self.cursors[pos]] >= length):
                self.cursors[pos] += 1

This data structure allows us to count efficiently how many events are in all time windows: how many times the student attempted this skill in the last hour, day, week, month. So, we need one queue per student and per category (attempt, success).

In [4]:
q = OurQueue()
q.push(0)  # New event at t = 0 s
q.push(0.8 * 3600 * 24)  # New event at 80% of one day
q.push(5 * 3600 * 24)  # t = 5 days
q.get_counters(5 * 3600 * 24)

[3, 3, 3, 1, 1]

So there are 3 events in total, 3 within the last month, 3 within the last week, 1 within the last day and 1 within the last hour.


In [5]:
q.push(40 * 3600 * 24)  # t = 40 days
q.get_counters(40 * 3600 * 24)

[4, 1, 1, 1, 1]

Now time flies, there are 4 events since forever, 1 within the last month, week and day.

A more complex example:

In [7]:
q = OurQueue()
q.push(0)
q.push(10)
q.push(3599)
q.push(3600)
q.push(3601)
q.push(3600 * 24)
q.push(3600 * 24 + 1)
q.push(3600 * 24 * 7)
q.push(3600 * 24 * 7 + 1)
q.push(3600 * 24 * 7 * 30)
q.push(3600 * 24 * 7 * 30 + 1)
q.get_counters(3600 * 24 * 7 * 30 + 1)

[11, 2, 2, 2, 2]

In [8]:
q.get_counters(3600 * 24 * 7 * 30 * 2)  # Way later

[11, 0, 0, 0, 0]

Gosh, this student has not practiced for a while! All counters (except forever) are zero.

In DAS3H, these counters are used as precious features for either running logistic regression or factorization machines.